In [ ]:
!pip install cdqa 
!pip install pytorch_pretrained_bert

In [4]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sijwa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
## Function to convert text corpus to para
def paraconverion(text):
    x1 = sent_tokenize(text)
    l=0
    gr=[]
    fin=[]
    for x in x1:
        t = word_tokenize(x)
        s = len(t)
        l=l+s
        if l<=480:
            gr.append(x)
        else:
            listToStr = ' '.join(map(str, gr))
            fin.append(listToStr)
            gr=[]
            gr.append(x)

            l=s

    listToStr = ' '.join(map(str, gr))
    fin.append(listToStr)
    return fin

In [6]:
import os
os.chdir('C:\\Term3\\Capstone\\Data\\clean text\\Final Demo')

In [8]:
##firsts point to the directory where text files are kept

import glob
df2 = pd.DataFrame(columns =['title','paragraphs'])
i=0
for file in sorted(glob.glob("*.txt")):
    with open(file,'r') as f:
        text = f.read()
    fin=paraconverion(text)
    name=file.split('.')[0]
    df2.loc[i]=[name]+[fin]
    i=i+1

In [10]:
df2['paragraphs'][0]

["What do we do in clustering? Grouping homogeneous data points, exactly we group homogeneous data to create a cluster right. This is performed generally as part of EDA, and we might have to do unsupervised learning techniques that is clustering, before we do any supervised learning techniques. Now when we do clustering, if it is a single variable, no problem, you can just segregated it. If it is 2 variables then we can visualise as well one and two variables not an issue, but when we have greater than two variables, that's when the problem occurs. So when we have a multi variable scenario we use the two techniques which are basically called as hierarchical and non hierarchical. Non hierarchical is also known as K Means. These are the two techniques. What is another name for Hierarchical Clustering? Agglomerative Clustering. What all different techniques that we use for measuring distances. OK, let me let me take it in a stepwise. So what is the process? How do we start hierarchical cl

In [ ]:
## First change the directory to point to this pretrained model
cdqa_pipeline = QAPipeline(reader='distilbert_qa_trained1.joblib')

In [ ]:
cdqa_pipeline.fit_retriever(df2)

In [ ]:
flag=True
print("CHATTY: My name is Chatty. If you want to exit, type Bye!")
while(flag==True):
    import warnings
    warnings.filterwarnings("ignore")
    user_response = input()
    user_response=user_response.lower()
    #resp=[user_response]
    #pred=list(nb.predict(resp))[0]

    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("CHATTY: You are welcome..")
        else:
            print("CHATTY: ",end="")
            prediction = cdqa_pipeline.predict(user_response)
                
            print("Answer: "+prediction[0])
    else:
        flag=False
        print("CHATTY: Take Care, Remember social distancing!!")

In [ ]:
## for Flask app frontend via Google chrome

!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from flask import Flask
import random
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def index():
    return render_template("index-chat.html")

@app.route("/get")
#function for the bot response
def get_bot_response():
    userText = request.args.get('msg')
    if(userText!='bye'):
        if(userText=='thanks' or userText=='thank you' ):
            return("You are welcome..")
        else:
            if userText in greetings:
                random_greeting = random.choice(greetings)
                return(random_greeting)
            elif userText in question:
                return('I am fine')
            elif (userText=='Yes')|(userText=='yes'):
                return('Great ')
            elif (userText=='No')|(userText=='no'):
                return('no worries, I will ask tutor to connect with you, Thanks')
            else:
                prediction = cdqa_pipeline.predict(userText)             
                return str(prediction[0])
               

    else:
        return("Take Care, Remember social distancing!!")
if __name__ == "__main__":
    app.run()

In [ ]:
## for flask app function in local

from flask import Flask, render_template, request
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index-chat.html")

@app.route("/get")
#function for the bot response
def get_bot_response():
    userText = request.args.get('msg')
    if(userText!='bye'):
        if(userText=='thanks' or userText=='thank you' ):
            return("You are welcome..")
        else:
            prediction = cdqa_pipeline.predict(userText)
            return str(prediction[0])
    else:
        return("Take Care, Remember social distancing!!")
if __name__ == "__main__":
    app.run()

In [ ]:
## create training dataset from the text corpus, output is json file which is used by cdQA annptator front end to create training questions.

from cdqa.utils.converters import df2squad
##Put a path in output_dir parameter
json_data = df2squad(df=df2, squad_version='v1.1', output_dir='.', filename='dataset-name.json')

In [ ]:
## Train Model and save trained model
path_to_data = '/content/drive/My Drive/Clean2/Training Data.json' ## Path where trained json file is kept
cdqa_pipeline.fit_reader(path_to_data) ## Run training of model
cdqa_pipeline.dump_reader('bert_qa_trained.joblib') ## Save the trained model to required path